In [2]:
import os
import pandas as pd
import numpy as np
import yfinance as yf
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime

sns.set_theme(style="whitegrid")


#===============================================================================
# 1) LOAD S&P 500 TICKERS (INCLUDING ^GSPC) & CLEAN SYMBOLS
#===============================================================================
url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
sp500_table = pd.read_html(url, header=0)[0]
sp500_table.drop_duplicates(subset=["Symbol"], inplace=True)

sp500_tickers = sp500_table["Symbol"].unique().tolist()

# Replace '.' with '-' for yfinance (e.g., BRK.B -> BRK-B)
sp500_tickers = [t.replace('.', '-') for t in sp500_tickers]

# Some tickers might still fail if Yahoo truly has no data (e.g. BF-B, BRK-B).
# Optionally remove them if needed:
bad_tickers = ["BF-B","BRK-B"]  # if they keep failing, you can exclude
# But let's try first without removing them. If you see repeated errors, uncomment:
# sp500_tickers = [t for t in sp500_tickers if t not in bad_tickers]

all_tickers = sp500_tickers + ["^GSPC"]
print(f"Total S&P 500 tickers (cleaned): {len(sp500_tickers)}")


#===============================================================================
# 2) SINGLE 10-YEAR DOWNLOAD WITH auto_adjust=False & CACHING
#===============================================================================
cache_file = "sp500_10y.pkl"
if os.path.exists(cache_file):
    print(f"Loading cached data from {cache_file}")
    all_data = pd.read_pickle(cache_file)
else:
    print("Downloading 10-year daily data for all S&P 500 + ^GSPC...")
    all_data = yf.download(
        tickers=all_tickers,
        period="10y",
        interval="1d",
        group_by="ticker",
        progress=False,
        auto_adjust=False  # Force "Close" & "Adj Close" columns to exist
    )
    all_data.to_pickle(cache_file)
    print(f"Data cached to {cache_file}")

# If "Close" is present, use it; else fallback to "Adj Close"
if "Close" in all_data.columns.levels[0]:
    df_close = all_data["Close"]
else:
    df_close = all_data["Adj Close"]

print("Data shape (close prices):", df_close.shape)


#===============================================================================
# 3) SECTOR-BASED MEDIAN RETURNS & PLOTS
#===============================================================================
# Convert to long format for sector-based analysis
df_long = (
    df_close
    .stack()          # merges Ticker into a new level
    .reset_index()
    .rename(columns={"level_1": "Ticker", 0: "Close"})
)

# Merge sector info
df_long = df_long.merge(
    sp500_table[["Symbol", "GICS Sector"]],
    left_on="Ticker",
    right_on="Symbol",
    how="left"
)
df_long.drop(columns=["Symbol"], inplace=True)

# Sort by date
df_long["Date"] = pd.to_datetime(df_long["Date"])
df_long.sort_values(["Date", "Ticker"], inplace=True)

# Compute daily returns
df_long["Daily_Return"] = df_long.groupby("Ticker")["Close"].pct_change()

# Separate index ^GSPC
df_index = df_long[df_long["Ticker"] == "^GSPC"].copy()
df_stocks = df_long[df_long["Ticker"] != "^GSPC"].copy()

# Median daily return by (Date, Sector)
sector_median_returns = (
    df_stocks
    .groupby(["Date", "GICS Sector"])["Daily_Return"]
    .median()
    .dropna()
    .reset_index()
    .pivot(index="Date", columns="GICS Sector", values="Daily_Return")
)

# Growth of $1
sector_value = (1 + sector_median_returns).cumprod()

# S&P 500 index
df_index.sort_values("Date", inplace=True)
df_index["Index_Return"] = df_index["Close"].pct_change()
df_index["Index_Value"] = (1 + df_index["Index_Return"]).cumprod()

# Plot: Growth of $1 by sector (median) vs S&P 500
plt.figure(figsize=(12, 8))
colors = sns.color_palette("husl", n_colors=len(sector_value.columns))

for i, sector in enumerate(sector_value.columns):
    plt.plot(sector_value.index, sector_value[sector], label=sector, color=colors[i])

plt.plot(df_index["Date"], df_index["Index_Value"], label="S&P 500 (^GSPC)", color="black", linewidth=2)
plt.title("Growth of $1 Over 10 Years (Median Sector Returns vs. S&P 500)")
plt.xlabel("Date")
plt.ylabel("Value of $1 (Median Returns)")
plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left")
plt.tight_layout()
plt.show()

# Final sector values
final_sector_values = sector_value.iloc[-1].sort_values(ascending=False)
final_index_value = df_index["Index_Value"].iloc[-1]
print("\n=== Final Value of $1 Invested (Median Sector) ===")
print(final_sector_values)
print(f"\nS&P 500 Final Value of $1 Invested: {final_index_value:.2f}")

# Sector median volatility
ticker_vol = (
    df_stocks
    .groupby("Ticker")["Daily_Return"]
    .std()
    .reset_index()
    .rename(columns={"Daily_Return": "Volatility"})
)
ticker_sector = df_stocks.drop_duplicates(subset=["Ticker"])[["Ticker", "GICS Sector"]]
ticker_vol = ticker_vol.merge(ticker_sector, on="Ticker", how="left")
median_vol_by_sector = ticker_vol.groupby("GICS Sector")["Volatility"].median()

final_median_cum_return = sector_value.iloc[-1]
scatter_df = pd.DataFrame({
    "Sector": final_median_cum_return.index,
    "Median_Cum_Return": final_median_cum_return.values,
    "Median_Volatility": median_vol_by_sector
}).dropna()

# Scatter plot: Sector Risk vs Return
plt.figure(figsize=(10, 6))
sns.scatterplot(
    data=scatter_df,
    x="Median_Volatility",
    y="Median_Cum_Return",
    hue="Sector",
    palette="husl",
    s=150
)
for i in range(len(scatter_df)):
    plt.text(
        x=scatter_df["Median_Volatility"].iloc[i] + 0.0001,
        y=scatter_df["Median_Cum_Return"].iloc[i],
        s=scatter_df["Sector"].iloc[i],
        fontsize=9
    )
plt.title("Sector Risk vs. Return (Median-Based)")
plt.xlabel("Median Volatility (Std Dev of Daily Returns)")
plt.ylabel("Final Median Cumulative Return (Decimal)")
plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left")
plt.tight_layout()
plt.show()


#===============================================================================
# 4) MULTI-TIME-FRAME FOR TECH TICKERS ONLY (10y, 5y, 3y, 1y)
#===============================================================================
# Identify Tech tickers
tech_df = sp500_table[sp500_table['GICS Sector'] == 'Information Technology']
raw_tech_tickers = tech_df["Symbol"].unique().tolist()
tech_tickers_yf = [t.replace('.', '-') for t in raw_tech_tickers if t != "^GSPC"]

# Extract columns that exist
available_cols = [col for col in df_close.columns if col in tech_tickers_yf]
tech_close = df_close[available_cols].copy()

def compute_metrics(price_df):
    results = []
    for ticker in price_df.columns:
        s = price_df[ticker].dropna()
        if s.empty:
            continue
        cum_return = (s.iloc[-1] / s.iloc[0]) - 1
        daily_ret = s.pct_change().dropna()
        vol = daily_ret.std() * np.sqrt(252)
        ratio = np.nan if vol == 0 else (cum_return / vol)
        results.append({
            "Ticker": ticker,
            "Cumulative Return": cum_return,
            "Volatility": vol,
            "Return-to-Risk": ratio
        })
    return pd.DataFrame(results)

# Subset date ranges
df_10y_tech = tech_close.loc["2012-01-01":"2022-01-01"]
df_5y_tech  = tech_close.loc["2017-01-01":"2022-01-01"]
df_3y_tech  = tech_close.loc["2019-01-01":"2022-01-01"]
df_1y_tech  = tech_close.loc["2021-01-01":"2022-01-01"]

r10y = compute_metrics(df_10y_tech).rename(columns={
    "Cumulative Return": "CumulativeReturn_10y",
    "Volatility":        "Volatility_10y",
    "Return-to-Risk":    "ReturnToRisk_10y"
})
r5y = compute_metrics(df_5y_tech).rename(columns={
    "Cumulative Return": "CumulativeReturn_5y",
    "Volatility":        "Volatility_5y",
    "Return-to-Risk":    "ReturnToRisk_5y"
})
r3y = compute_metrics(df_3y_tech).rename(columns={
    "Cumulative Return": "CumulativeReturn_3y",
    "Volatility":        "Volatility_3y",
    "Return-to-Risk":    "ReturnToRisk_3y"
})
r1y = compute_metrics(df_1y_tech).rename(columns={
    "Cumulative Return": "CumulativeReturn_1y",
    "Volatility":        "Volatility_1y",
    "Return-to-Risk":    "ReturnToRisk_1y"
})

df_10y = r10y[["Ticker","CumulativeReturn_10y","Volatility_10y","ReturnToRisk_10y"]]
df_5y  = r5y[ ["Ticker","CumulativeReturn_5y","Volatility_5y","ReturnToRisk_5y"]]
df_3y  = r3y[ ["Ticker","CumulativeReturn_3y","Volatility_3y","ReturnToRisk_3y"]]
df_1y  = r1y[ ["Ticker","CumulativeReturn_1y","Volatility_1y","ReturnToRisk_1y"]]

merged_tech = (
    df_10y.merge(df_5y, on="Ticker", how="inner")
          .merge(df_3y, on="Ticker", how="inner")
          .merge(df_1y, on="Ticker", how="inner")
)

print("merged_tech shape:", merged_tech.shape)

# Standardize
def zscore(series):
    return (series - series.mean()) / series.std(ddof=0)

cumret_cols = ["CumulativeReturn_10y","CumulativeReturn_5y","CumulativeReturn_3y","CumulativeReturn_1y"]
vol_cols    = ["Volatility_10y","Volatility_5y","Volatility_3y","Volatility_1y"]
rtr_cols    = ["ReturnToRisk_10y","ReturnToRisk_5y","ReturnToRisk_3y","ReturnToRisk_1y"]

for col in cumret_cols + vol_cols + rtr_cols:
    if merged_tech[col].std(ddof=0) == 0:
        merged_tech["z_"+col] = 0
    else:
        merged_tech["z_"+col] = zscore(merged_tech[col])

for col in vol_cols:
    merged_tech["z_"+col] = -1.0 * merged_tech["z_"+col]

# Weighted sum approach
metric_weights = {
    "CumulativeReturn": 0.4,
    "Volatility":       0.3,
    "ReturnToRisk":     0.3
}
time_weights = {
    "10y": 0.4,
    "5y":  0.3,
    "3y":  0.2,
    "1y":  0.1
}

def compute_final_score(row):
    score = 0.0
    for tf in ["10y","5y","3y","1y"]:
        cr_col  = f"z_CumulativeReturn_{tf}"
        vol_col = f"z_Volatility_{tf}"
        rtr_col = f"z_ReturnToRisk_{tf}"
        tf_sum = (metric_weights["CumulativeReturn"] * row[cr_col]
                  + metric_weights["Volatility"]       * row[vol_col]
                  + metric_weights["ReturnToRisk"]     * row[rtr_col])
        score += time_weights[tf] * tf_sum
    return score

merged_tech["FinalScore"] = merged_tech.apply(compute_final_score, axis=1)
merged_tech.sort_values("FinalScore", ascending=False, inplace=True)

top_20_tech = merged_tech.head(20).copy()
print("\n=== Top 20 Tech Stocks (Multi-Time-Frame Weighted Sum) ===")
print(top_20_tech[["Ticker","FinalScore"]])

# Quick barplot
top_20_sorted = top_20_tech.sort_values("FinalScore", ascending=True)
plt.figure(figsize=(8, 6))
sns.barplot(
    data=top_20_sorted,
    x="FinalScore",
    y="Ticker",
    palette="viridis"
)
plt.title("Top 20 Tech Stocks (Multi-Time-Frame Weighted Sum)")
plt.xlabel("Final Score")
plt.ylabel("Ticker")
plt.tight_layout()
plt.show()



Total S&P 500 tickers (cleaned): 503
Loading cached data from sp500_10y.pkl


KeyError: 'Adj Close'